In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.preprocessing.image import ImageDataGenerator

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(r'C:\Users\Tanmay\Downloads\intel\seg_train\seg_train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(r'C:\Users\Tanmay\Downloads\intel\seg_test\seg_test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

In [5]:
# Part 2 - Building the CNN
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', strides=2, input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening 
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))


# for mulitclassification
cnn.add(Dense(6, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='softmax'))
cnn.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])

In [6]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 7

In [7]:


# Training the CNN on the Training set and evaluating it on the Test set
r=cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
439/439 [==============================] - 29s 65ms/step - loss: 1.1306 - accuracy: 0.5645 - val_accuracy: 0.5710 - val_loss: 1.1106
Epoch 2/15
439/439 [==============================] - 20s 45ms/step - loss: 1.0655 - accuracy: 0.6457 - val_accuracy: 0.6547 - val_loss: 1.0586
Epoch 3/15
439/439 [==============================] - 20s 45ms/step - loss: 1.0406 - accuracy: 0.6831 - val_accuracy: 0.6270 - val_loss: 1.0735
Epoch 4/15
439/439 [==============================] - 22s 49ms/step - loss: 1.0228 - accuracy: 0.7141 - val_accuracy: 0.7117 - val_loss: 1.0241
Epoch 5/15
439/439 [==============================] - 21s 49ms/step - loss: 1.0130 - accuracy: 0.7282 - val_accuracy: 0.7173 - val_loss: 1.0199
Epoch 6/15
439/439 [==============================] - 21s 49ms/step - loss: 1.0036 - accuracy: 0.7436 - val_accuracy: 0.7480 - val_loss: 1.0011
Epoch 7/15
439/439 [==============================] - 27s 61ms/step - loss: 0.9942 - accuracy: 0.7574 - val_accuracy: 0.7280 - val_loss:

In [10]:
classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [35]:
from tensorflow.keras.models import load_model

cnn.save('intel classification SVM.h5') #Got 80% accuracy with SVM

In [30]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('intel classification SVM.h5')

In [31]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img(r'C:\Users\Tanmay\Downloads\intel\seg_pred\seg_pred\360.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

In [32]:
result

array([[5.4675713e-02, 2.6780053e-04, 4.5118030e-02, 1.2738255e-01,
        3.6364514e-02, 7.3619139e-01]], dtype=float32)

In [33]:
np.argmax(result)

5

In [34]:
classes[np.argmax(result)]

'street'